In [1]:
import json
import numpy as np
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

In [3]:
with open("items.json") as f:
    items = json.load(f)

In [44]:
parent = {}

In [45]:
num_cat = len(items)
cat_id = 0
category_in_mapping = {}
category_out_mapping = {}
for category in items:
    category_in_mapping[cat_id] = category["Kategori"]
    category_out_mapping[category["Kategori"]] = cat_id
    if category["Ana kategori"] in parent:
        parent[category["Ana kategori"]].append(category["Kategori"])
    else:
        parent[category["Ana kategori"]] = [category["Kategori"]]
    cat_id += 1

In [46]:
parent

{'Gıda, Şekerleme': ['Şekersiz Tatlandırıcılı Ürünler',
  'Sıvı Yağlar',
  'Çikolata, Gofret',
  'Şeker',
  'Bisküvi, Çerez',
  'Hazır Yemek, Konserve, Salça',
  'Meze',
  'Çorba Ve Bulyonlar',
  'Tuz, Baharat, Harç',
  'Sağlıklı Yaşam Ürünleri',
  'Unlu Mamül, Tatlı',
  'Sakız, Şekerleme',
  'Dondurulmuş Gıda',
  'Bakliyat, Makarna'],
 'Deterjan, Temizlik': ['Çamaşır Gereçleri',
  'Ev Temizlik Gereçleri',
  'Banyo Gereçleri',
  'Bulaşık Yıkama',
  'Ev Temizlik Ürünleri',
  'Çamaşır Yıkama'],
 'Süt, Kahvaltılık': ['Yoğurt',
  'Süt',
  'Yumurta',
  'Dondurma',
  'Peynir',
  'Sütlü Tatlı, Krema',
  'Tereyağ, Margarin',
  'Kahvaltılıklar'],
 'İçecek': ['Gazlı İçecekler',
  'Çay, Kahve',
  'Günlük İçecek',
  'Sular',
  'Maden Suları',
  'Gazsız İçecekler'],
 'Kağıt, Kozmetik': ['Hijyenik Pedler',
  'Makyaj Ve Süs Ürünleri',
  'Ağdalar, Tüy Dökücüler',
  'Tıraş Malzemeleri',
  'Sağlık Ürünleri',
  'Vücut, Cilt Bakım',
  'Ağız Bakım Ürünleri',
  'Kağıt Ürünleri',
  'Parfüm, Deodorant',
  'Du

In [30]:
print(category_in_mapping)

{0: 'Şekersiz Tatlandırıcılı Ürünler', 1: 'Sıvı Yağlar', 2: 'Çamaşır Gereçleri', 3: 'Yoğurt', 4: 'Çikolata, Gofret', 5: 'Gazlı İçecekler', 6: 'Süt', 7: 'Çay, Kahve', 8: 'Ev Temizlik Gereçleri', 9: 'Banyo Gereçleri', 10: 'Hijyenik Pedler', 11: 'Anne Ürünleri', 12: 'Kırmızı Et', 13: 'Makyaj Ve Süs Ürünleri', 14: 'Günlük İçecek', 15: 'Şeker', 16: 'Bisküvi, Çerez', 17: 'Yumurta', 18: 'Hazır Yemek, Konserve, Salça', 19: 'Meze', 20: 'Sular', 21: 'Kişisel İlgi, Eğlence', 22: 'Dondurma', 23: 'Çorba Ve Bulyonlar', 24: 'Ağdalar, Tüy Dökücüler', 25: 'Peynir', 26: 'Bulaşık Yıkama', 27: 'Bebek Ürünleri', 28: 'Meyve', 29: 'Tıraş Malzemeleri', 30: 'Bahçe, Çiçek, Kamp, Piknik', 31: 'Sağlık Ürünleri', 32: 'Kitap, Dergi, Kırtasiye', 33: 'Tuz, Baharat, Harç', 34: 'Elektrik, Elektronik', 35: 'Oyuncaklar', 36: 'Ev Temizlik Ürünleri', 37: 'Vücut, Cilt Bakım', 38: 'Ağız Bakım Ürünleri', 39: 'Çamaşır Yıkama', 40: 'Kağıt Ürünleri', 41: 'Bebek Bezi', 42: 'Kümes Hayvanları', 43: 'Sütlü Tatlı, Krema', 44: 'Parfüm

In [5]:
X = []
Y = []
for category in items:
    y = np.zeros(num_cat)
    y[category_out_mapping[category["Kategori"]]] = 1
    for item in category["Item"]:
        x = item[0]
        x_vec = np.zeros(400)
        count = 0
        for word in x.lower().split():
            if word in word_vectors:
                count+=1
                x_vec += word_vectors[word]
        if count==0:
            continue
        else:
            X.append(x_vec/count)
            Y.append(category_out_mapping[category["Kategori"]])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [23]:
clf = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000000).fit(X, Y)

In [25]:
clf.score(X, Y)

0.8946085897045385

In [29]:
import pickle
filename = 'classifier.sav'
pickle.dump(clf, open(filename, 'wb'))

In [14]:
def pred(tag, threshold=0.65):
    c = 0
    x = np.zeros(400)
    for w in tag.lower().split():
        if w in word_vectors:
            c += 1
            x += word_vectors[w]
    x = x/c
    probs = clf.predict_proba(x.reshape(1,-1))
    if max(probs[0])>threshold:
        return category_in_mapping[np.argmax(probs)]
    else:
        return None

In [21]:
pred("rakı")

'Bisküvi, Çerez'

In [12]:
for i in range(20):
    tp = sum([x>i*0.05 for x in correct])
    fn = sum([x>i*0.05 for x in wrong])
    print(i*0.05, tp/(tp+fn), tp/len(correct))

0.0 0.7764240024367957 1.0
0.05 0.7764240024367957 1.0
0.1 0.7773711497407746 1.0
0.15000000000000002 0.7821143208358943 0.9984307571596704
0.2 0.7897035881435257 0.9929384072185171
0.25 0.8026905829596412 0.9831306394664574
0.30000000000000004 0.8201320132013201 0.9748921145547274
0.35000000000000003 0.8349282296650717 0.9584150647312671
0.4 0.8496639547223205 0.9423303256178893
0.45 0.8628613787991104 0.9132993330717929
0.5 0.8759208995734781 0.8862298940761083
0.55 0.8909606809890556 0.8622989407610828
0.6000000000000001 0.9053533190578158 0.8293448411141624
0.65 0.914568345323741 0.7979599843075716
0.7000000000000001 0.9213911386374464 0.7587289132993331
0.75 0.929779600205023 0.7116516280894468
0.8 0.9368421052631579 0.6633974107493135
0.8500000000000001 0.9433846153846154 0.6014123185562966
0.9 0.954512105649303 0.5103962338171832
0.9500000000000001 0.96 0.3954491957630443
